In [1]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

In [2]:
table26 = pd.read_excel('Table 26.xlsx')
ratio_length_depth = pd.read_excel('Length depth ratio.xlsx')
table27 = pd.read_excel('Table 27.xlsx')
table19 = pd.read_excel('Table 19.xlsx')
clause40211 = pd.read_excel('clause 40_2_1_1.xlsx')
clause26_2_1_1 = pd.read_excel('clause 26_2_1_1.xlsx')

In [3]:
def reoccur():
    repeat = input("Enter y or Y for calculating again: ")
    if(repeat == 'y' or repeat == 'Y'):
        main()     
    else:
        print("Thank you for using this calculator")
def main():
    user_options = {1:"initial depth", 2:"depth with mortar and rod", 3:"total load and factorial load", 4:"alphas", 5:"moment", 6:"Ast",7:"spacing"}
    option = int(input(f"enter number coresponding to value you want from {user_options}"))
    #creating cases to use formula for different situations
    # finding out depth of concrete required by first calculating depth 
    if(option==1):
        length_room = float(input("Enter the smaller side of room in meter: "))*1000
        case = int(input(f"from table: \n {ratio_length_depth} \n enter case index (0-8):"))
        depth_concrete_initial = (length_room / ratio_length_depth.iloc[case,1])
        print(f"depth without considering mortar or rod diameter is: {depth_concrete_initial}mm")
    #calculating depth with mortar and rod
    elif(option==2):
        length_room = float(input("Enter the smaller side of room in meter: "))*1000
        diameter_rod = float (input("Enter the diameter of rod required in mm: "))
        case = int(input(f"from table: \n {ratio_length_depth} \n enter case index (0-8):"))
        depth_concrete_initial = (length_room / ratio_length_depth.iloc[case,1])
        depth_concrete = depth_concrete_initial + 15 + (diameter_rod/2)
        print(f"Depth we get after taking rod and mortar in consideration is {depth_concrete} mm")
    #calcultaing total load and factorial load which would be helpful further
    elif(option==3):
        length_room = float(input("Enter the smaller side of room in meter: "))*1000
        diameter_rod = float (input("Enter the diameter of rod required in mm: "))
        floor_finish = float(input("enter the floor finish pressure(in KN/m*m): "))
        live_load = float(input("enter the live load to be considered(in KN/m*m): "))
        case = int(input(f"from table: \n {ratio_length_depth} \n enter case index (0-8):"))
        depth_concrete_initial = (length_room / ratio_length_depth.iloc[case,1])
        depth_concrete = depth_concrete_initial + 15 + (diameter_rod/2)
        total_load = floor_finish * 1 + live_load * 1 + 25*1*(depth_concrete/1000)
        factorial_load = 1.5 * total_load
        print(f"Total load is {total_load}kN/m and Factorial load is {factorial_load}kN/m considering 1.5 as saftey factor")
    #calculating alphas
    elif(option==4):
        length_room = float(input("Enter the smaller side of room in meter: "))*1000
        breadth_room = float(input("Enter the bigger side of room in meter: "))*1000
        diameter_rod = float (input("Enter the diameter of rod required in mm: "))
        depth_concrete_initial = float (input("Enter the initial depth of concrete you obtained without considering mortar or rod in mm: "))
        case = int(input(f"from table: \n {ratio_length_depth} \n enter case index (0-8):"))
        length_ex = length_room + depth_concrete_initial
        breadth_ey = breadth_room + depth_concrete_initial - diameter_rod
        a = (breadth_ey)/(length_ex)
        def information(a):
            if(a<=1.5):
                index = int(np.floor((a-1)*10))
                alpha_x_negative = table26.iloc[2*case + 1,index+2] + ((table26.iloc[2*case +1,index+3] - table26.iloc[2*case + 1,index+2])/0.1)*(a - (1 + (index/10)))
                alpha_x_positive = table26.iloc[2*case + 2, index+2] + ((table26.iloc[2*case + 2,index+3] - table26.iloc[2*case + 2,index+2])/0.1)*(a - (1 + (index/10)))
                alpha_y_negative = table26.iloc[2*case + 1,10]
                alpha_y_positive = table26.iloc[2*case + 2,10]
            elif(a>1.5 and a<1.75):
                index = 7
                alpha_x_negative = table26.iloc[2*case + 1,index+2] + ((table26.iloc[2*case +1,index+3] - table26.iloc[2*case + 1,index+2])/0.25)*(a - (1.5))
                alpha_x_positive = table26.iloc[2*case + 2, index+2] + ((table26.iloc[2*case + 2,index+3] - table26.iloc[2*case + 2,index+2])/0.25)*(a - (1.5))
                alpha_y_negative = table26.iloc[2*case + 1,10]
                alpha_y_positive = table26.iloc[2*case + 2,10]
            else:
                index = 8
                alpha_x_negative = table26.iloc[2*case + 1,index+2] + ((table26.iloc[2*case +1,index+3] - table26.iloc[2*case + 1,index+2])/0.25)*(a - (1.75))
                alpha_x_positive = table26.iloc[2*case + 2, index+2] + ((table26.iloc[2*case + 2,index+3] - table26.iloc[2*case + 2,index+2])/0.25)*(a - (1.75))
                alpha_y_negative = table26.iloc[2*case + 1,10]
                alpha_y_positive = table26.iloc[2*case + 2,10]
            alpha_max = max(alpha_x_positive,alpha_x_negative,alpha_y_positive,alpha_y_negative)
            return (alpha_max,alpha_x_positive,alpha_x_negative,alpha_y_positive,alpha_y_negative)
        if(a<2):
            #storing value of aplha
            alpha_max,alpha_x_positive,alpha_x_negative,alpha_y_positive,alpha_y_negative = information(a)
            print(f"maximum alpha is {alpha_max}, alpha x+ is {alpha_x_positive}, alpha x- is {alpha_x_negative}, alpha y+ is {alpha_y_positive}, alpha y- is {alpha_y_negative}")
        else:
            print("Take length breadth correctly as their modified ratio is greater than 2 and we can't calculate for such ratio")
    # calculating moment 
    elif(option==5):
        alpha_max,alpha_x_positive,alpha_x_negative,alpha_y_positive,alpha_y_negative = [float(x) for x in input("Enter maximum alpha,alpha x+,alpha x-,alpha y+,alpha y- in same order with comma: ").split(',')]
        factorial_load = float(input("Factorial load you obtained before in kN/m: "))
        length_room = float(input("Enter the smaller side of room in meter: "))*1000
        depth_concrete_initial = float (input("Enter the initial depth of concrete you obtained without considering mortar or rod in mm: "))
        length_ex = length_room + depth_concrete_initial
        moment_x_positive = alpha_x_positive * factorial_load * (length_ex**2)
        moment_x_negative = alpha_x_negative * factorial_load * (length_ex**2)
        moment_y_positive = alpha_y_positive * factorial_load * (length_ex**2)
        moment_y_negative = alpha_y_negative * factorial_load * (length_ex**2)
        moment_max = alpha_max * factorial_load * (length_ex**2)
        print(f"maximum moment is {moment_max *(10**(-6))}kNm, moment x+ is {moment_x_positive *(10**(-6))}kNm, moment x- is {moment_x_negative *(10**(-6))}kNm, moment y+ is {moment_y_positive *(10**(-6))}kNm, moment y- is {moment_y_negative *(10**(-6))}kNm")
    #calculation of Ast
    elif(option==6):
        diameter_rod = float(input("diameter of rod you want to use in mm: "))
        fck = float(input("enter the fck value(in N/mm*mm) (20, 25, 30, 35, 40): "))
        fy = float(input("enter the fy value(250, 415, 500): "))
        depth_concrete_initial = float (input("Enter the initial depth of concrete you obtained without considering mortar or rod in mm: "))
        depth_concrete = float (input("Enter the initial depth of concrete you obtained  considering mortar or rod in mm: "))
        # sachin ko print karana h
        Ast_minimum = (.12/100)*(1000)*(depth_concrete)
        moment_x_positive, moment_x_negative, moment_y_positive, moment_y_negative = [float(x) for x in input("Enter moment x+,moment x-,moment y+,moment y- in kNm in same order with comma:").split(',')]
        moment_x_positive, moment_x_negative, moment_y_positive, moment_y_negative = moment_x_positive*(10 ** (6)), moment_x_negative*(10 ** (6)), moment_y_positive*(10 ** (6)), moment_y_negative*(10 ** (6)) 
        Ast_x_positive = ((0.87*fy*depth_concrete_initial)- np.sqrt(((0.87*fy*depth_concrete_initial)**2) - ((4*moment_x_positive*0.87*fy*fy)/(fck*1000))))/((2*0.87*fy*fy)/(fck*1000))
        Ast_x_negative = ((0.87*fy*depth_concrete_initial)- np.sqrt(((0.87*fy*depth_concrete_initial)**2) - ((4*moment_x_negative*0.87*fy*fy)/(fck*1000))))/((2*0.87*fy*fy)/(fck*1000))
        Ast_y_positive = ((0.87*fy*depth_concrete_initial)- np.sqrt(((0.87*fy*depth_concrete_initial)**2) - ((4*moment_y_positive*0.87*fy*fy)/(fck*1000))))/((2*0.87*fy*fy)/(fck*1000))
        Ast_y_negative = ((0.87*fy*depth_concrete_initial)- np.sqrt(((0.87*fy*depth_concrete_initial)**2) - ((4*moment_y_negative*0.87*fy*fy)/(fck*1000))))/((2*0.87*fy*fy)/(fck*1000))
        Ast_individual = (np.pi) * ((diameter_rod / 2)**2)
        print(f"Ast x+ is {Ast_x_positive}mm*mm, Ast x- is {Ast_x_negative}mm*mm, Ast y+ is {Ast_y_positive}mm*mm, Ast y- is {Ast_y_negative}mm*mm")
    #calculating spacing
    else:
        Ast_x_positive,Ast_x_negative,Ast_y_positive,Ast_y_negative,Ast_individual = [float(x) for x in input("Enter Ast x+,Ast x-,Ast y+,Ast y-,Ast individual rod in sq. mm in same order with comma after you check and have modified Asts':").split(',')]
        spacing_x = (1000)/(Ast_x_negative/Ast_individual)
        spacing_x_midspan = (1000)/(Ast_x_positive/Ast_individual)
        spacing_y = (1000)/(Ast_y_negative/Ast_individual)
        spacing_y_midspan = (1000)/(Ast_y_positive/Ast_individual)
        print(f"spacing x midspan is {spacing_x_midspan}mm, spacing y midspan is {spacing_y_midspan}mm, spacing x is {spacing_x}mm, spacing y is {spacing_y}mm")
    reoccur()
main()

    

enter number coresponding to value you want from {1: 'initial depth', 2: 'depth with mortar and rod', 3: 'total load and factorial load', 4: 'alphas', 5: 'moment', 6: 'Ast', 7: 'spacing'}1
Enter the smaller side of room in meter: 4
from table: 
                              Case Name  length/depth_concrete_initial
0                Four Edges Continuous                             32
1          One Short Edge Disontinuous                             32
2           One Long Edge Disontinuous                             32
3     Two Adjacent Edges Discontinuous                             30
4        Two Short Edges Discontinuous                             30
5         Two Long Edges Discontinuous                             30
6  Three Edges (shorter) Discontinuous                             28
7   Three Edges (longer) Discontinuous                             28
8             Four Edges Discontinuous                             28 
 enter case index (0-8):2
depth without considering m